# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298392715646                   -0.85    5.0
  2   -8.300198481565       -2.74       -1.26    1.6
  3   -8.300451196921       -3.60       -1.89    3.1
  4   -8.300462210436       -4.96       -2.71    2.1
  5   -8.300464134416       -5.72       -3.13    6.5
  6   -8.300464376479       -6.62       -3.28    4.8
  7   -8.300464497053       -6.92       -3.40    1.2
  8   -8.300464569916       -7.14       -3.53    1.0
  9   -8.300464633014       -7.20       -3.79    2.2
 10   -8.300464641205       -8.09       -3.96    1.0
 11   -8.300464642858       -8.78       -4.24    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67388417042                   -0.70    7.4
  2   -16.67797075924       -2.39       -1.14    1.0
  3   -16.67919959237       -2.91       -1.86    4.5
  4   -16.67926867837       -4.16       -2.68    2.6
  5   -16.67928417049       -4.81       -3.04    5.4
  6   -16.67928621012       -5.69       -3.47    2.0
  7   -16.67928622174       -7.94       -4.02    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32551456196                   -0.56    6.9
  2   -33.33275591074       -2.14       -1.00    1.1
  3   -33.33412582452       -2.86       -1.73    5.5
  4   -33.33426850114       -3.85       -2.64    2.8
  5   -33.33694262592       -2.57       -2.52   12.1
  6   -33.33694290058       -6.56       -2.56    1.6
  7   -33.33694385005       -6.02       -3.09    3.0
  8   -33.33694391357       -7.20       -3.56    5.2
  9   -33.33694392553       -7.92       -4.12    1.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298607840705                   -0.85    4.9
  2   -8.300272778587       -2.78       -1.59    1.2
  3   -8.300423274865       -3.82       -2.39    5.1
  4   -8.300318420791   +   -3.98       -2.17   13.8
  5   -8.300464485690       -3.84       -3.68    2.6
  6   -8.300464593206       -6.97       -3.94    2.4
  7   -8.300464632188       -7.41       -4.23    2.8


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.31622986773                   -0.56    6.2
  2   -33.32103547470       -2.32       -1.28    1.1
  3   -28.42985277674   +    0.69       -0.72    7.5
  4   -33.05608322832        0.67       -1.32    6.4
  5   -33.30877028634       -0.60       -1.53    4.8
  6   -33.32507646913       -1.79       -1.77    2.9
  7   -32.60030099469   +   -0.14       -1.13    5.2
  8   -33.32681862039       -0.14       -2.05    5.2
  9   -33.33661174604       -2.01       -2.43    2.5
 10   -33.33672293296       -3.95       -2.78    3.1
 11   -33.33683692540       -3.94       -2.89    2.4
 12   -33.33686485863       -4.55       -2.99    1.5
 13   -33.33692986238       -4.19       -3.32    2.6
 14   -33.33694205920       -4.91       -3.59    3.2
 15   -33.33694301035       -6.02       -3.72    2.6
 16   -33.33694380737       -6.10       -4.33    2.5


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.